In [1]:
import pandas as pd
import numpy as np
import nltk
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
import numpy as np
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('wordnet')
raw_data = pd.read_csv('./dataset.csv', sep=',')
for index in range(len(raw_data)):
    if raw_data.at[index, 'star_rating'] > 3:
        raw_data.at[index, 'star_rating'] = 1
    elif raw_data.at[index, 'star_rating'] == 3:
        raw_data.at[index, 'star_rating'] = 2
    else:
        raw_data.at[index, 'star_rating'] = 0
from bs4 import BeautifulSoup
import re
# data cleaning
raw_data['review_body'] = raw_data['review_body'].str.lower()
raw_data['review_body'] = raw_data['review_body'].apply(lambda x: re.sub(r'http\S+', '', BeautifulSoup(x, 'html.parser').get_text()))
def remove_non_alphabetical(text):
    return re.sub(r'[^a-zA-Z\s]', '', text)

raw_data['review_body'] = raw_data['review_body'].apply(remove_non_alphabetical)
raw_data['review_body'] = raw_data['review_body'].apply(lambda x: ' '.join(x.split()))
contraction_dict = contraction_dict = {
    "don't": "do not",
    "doesn't": "does not",
    "didn't": "did not",
    "can't": "cannot",
    "won't": "will not",
    "isn't": "is not",
    "haven't": "have not",
    "hasn't": "has not",
    "hadn't": "had not",
    "you're": "you are",
    "you've": "you have",
    "you'll": "you will",
    "when's": "when is",
    "let's": "let us",
    "'cause": "because",
    "shouldn't": "should not",
    "wouldn't": "would not",
    "couldn't": "could not",
    "wasn't": "was not",
    "weren't": "were not",
    "I'm": "I am",
    "I've": "I have",
    "I'll": "I will",
    "it's": "it is",
    "that's": "that is",
    "who's": "who is",
    "what's": "what is",
    "where's": "where is",
    "we're": "we are",
    "we've": "we have",
    "we'll": "we will",
    "they're": "they are",
    "they've": "they have",
    "they'll": "they will",
    "she's": "she is",
    "he's": "he is",
    "how's": "how is",
    "you'd": "you would",
    "we'd": "we would",
    "they'd": "they would",
}
raw_data['review_body'] = raw_data['review_body'].replace(contraction_dict, regex=True)
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
def remove_stop_words(text):
    words = text.split(" ")
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_words)
raw_data['review_body'] = raw_data['review_body'].apply(remove_stop_words)
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
nltk.download('omw-1.4') # I have to download this dataset
raw_data['review_body'] = raw_data['review_body'].apply(lambda x: ' '.join([lemmatizer.lemmatize(word) for word in x.split()]))
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
import nltk
import string
def preprocess_text(text):
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('english') + list(string.punctuation))
    tokens = [word.lower() for word in tokens if word.isalpha() and word.lower() not in stop_words]
    return tokens
raw_data['clean_reviw'] = raw_data['review_body'].apply(preprocess_text)

C:\Users\monkeydc\.conda\envs\561\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\monkeydc\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
C:\Users\monkeydc\.conda\envs\561\lib\site-packages\ipykernel_launcher.py:26: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
C:\Users\monkeydc\.conda\envs\561\lib\site-packages\ipykernel_launcher.py:26: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\User

In [2]:
import gensim.downloader as api
wv = api.load('word2vec-google-news-300')

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [4]:
from torch.utils.data import Dataset
class BetterDataset(Dataset):
    def __init__(self, reviews, labels, word2vec_model):
        self.reviews = reviews
        self.labels = labels
        self.word2vec_model = word2vec_model

    def __len__(self):
        return len(self.reviews)

    def __getitem__(self, idx):
        review = self.reviews[idx]
        # word_vectors = []

        concatenated_vectors = np.zeros((10, 300), dtype=np.float16)
        for i in range(min(10, len(review))):
            word = review[i]
            if word in self.word2vec_model.key_to_index:
                concatenated_vectors[i] = self.word2vec_model[word]
    
#         for i in range(10):
#             word = review[i]
#             if word in self.word2vec_model:
#                 word_vector = self.word2vec_model[word]
#             else:
#                 word_vector = torch.zeros(300, dtype=np.float16)  # Handle out-of-vocabulary words
#             word_vectors.append(word_vector)

        label = torch.tensor(self.labels[idx], dtype=torch.long)
        return torch.tensor(concatenated_vectors.flatten().tolist()), label

In [5]:
from sklearn.utils import shuffle
shuffled_dataset = shuffle(raw_data, random_state=42)
from sklearn.model_selection import train_test_split
X_train_g = shuffled_dataset[:200000]
X_test_g = shuffled_dataset[200000:]
print(len(X_train_g))
print(len(X_test_g))

200000
50000


In [6]:
X_train_g = X_train_g.reset_index(drop=True)

In [7]:
X_test_g = X_test_g.reset_index(drop=True)

In [8]:
train_dataset = BetterDataset(X_train_g['clean_reviw'], X_train_g['star_rating'], wv)

In [9]:
test_dataset = BetterDataset(X_test_g['clean_reviw'], X_test_g['star_rating'], wv)

In [10]:
class SentimentClassifier(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, output_size):
        super(SentimentClassifier, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size1).float() 
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size1, hidden_size2).float() 
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(hidden_size2, output_size).float()

    def forward(self, x):
        x = self.relu1(self.fc1(x))
        x = self.relu2(self.fc2(x))
        x = self.fc3(x)
        return x


input_size = 3000  
hidden_size1 = 50
hidden_size2 = 10
output_size_ternary = 3 

model_binary = SentimentClassifier(input_size, hidden_size1, hidden_size2, output_size_ternary).to(device)

criterion = nn.CrossEntropyLoss()
optimizer_binary = optim.Adam(model_binary.parameters(), lr=0.001)

In [11]:
batch_size = 16

In [12]:
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [13]:
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [14]:
num_epochs = 5
for epoch in range(num_epochs):
    model_binary.train()
    for inputs, labels in train_loader:
        inputs = inputs.view(inputs.size(0), -1).float()
        outputs_binary = model_binary(inputs.to(device))
        loss_binary = criterion(outputs_binary, labels.to(device))

        # Backward and optimize
        optimizer_binary.zero_grad()
        loss_binary.backward()
        optimizer_binary.step()

In [15]:
from sklearn.metrics import accuracy_score
model_binary.to('cpu')
model_binary.eval()
predictions_binary = []
true_labels_binary = []
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs = inputs.view(inputs.size(0), -1).float()
        outputs_binary = model_binary(inputs)
        _, predicted_binary = torch.max(outputs_binary, 1)
        predictions_binary.extend(predicted_binary.numpy())
        true_labels_binary.extend(labels.numpy())
accuracy_ternary = accuracy_score(true_labels_binary, predictions_binary)
print(accuracy_ternary)

0.62222
